In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import unicodedata
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.probability import FreqDist
from empath import Empath
import json
from scipy.spatial.distance import cdist

%matplotlib inline

df = pd.read_csv('data_prep.csv')
df

,title,author,school,century,sentence_lowered,sentence_words_count,normalized_sentence,sentiment_score,lexical_categories
0,plato - complete works,plato,plato,-4,"what's new, socrates, to make you leave your ...",23,new socrates make leave usual haunts lyceum sp...,-0.2960,"{'family': 1.0, 'vacation': 1.0, 'leisure': 1...."
1,plato - complete works,plato,plato,-4,surely you are not prosecuting anyone before t...,13,surely prosecuting anyone king archon,0.4404,"{'royalty': 1.0, 'optimism': 1.0, 'medieval': ..."
2,plato - complete works,plato,plato,-4,the athenians do not call this a prosecution b...,12,athenians call prosecution indictment euthyphro,-0.3346,"{'crime': 1.0, 'dispute': 1.0, 'stealing': 1.0..."
3,plato - complete works,plato,plato,-4,what is this you say?,5,NaN,0.0000,{}
4,plato - complete works,plato,plato,-4,"someone must have indicted you, for you are no...",19,someone indicted going tell indicted someone else,0.0000,"{'communication': 1.0, 'order': 1.0, 'speaking..."
...,...,...,...,...,...,...,...,...,...
360779,"women, race, and class",davis,feminism,20,but the socialization of housework including m...,22,socialization housework including meal prepara...,0.8462,"{'help': 1.0, 'money': 1.0, 'domestic_work': 2..."
360780,"women, race, and class",davis,feminism,20,the only significant steps toward endingdomest...,17,significant steps toward endingdomestic slaver...,-0.6124,"{'crime': 1.0, 'dispute': 1.0, 'government': 1..."
360781,"women, race, and class",davis,feminism,20,"working women, therefore, have a special and v...",14,working women special vital interest struggle ...,0.6808,"{'occupation': 1.0, 'weakness': 1.0, 'leisure'..."
360782,"women, race, and class",davis,feminism,20,"moreover, under capitalism, campaigns for jobs...",28,moreover capitalism campaigns jobs equal basis...,0.4939,"{'family': 1.0, 'play': 1.0, 'heroic': 1.0, 'f..."


In [3]:
grouped_df = df.groupby(['author', 'school']).size().reset_index(name='count')
sorted_df = grouped_df.sort_values('school')
sorted_df

,author,school,count
16,kripke,analytic,12479
31,russell,analytic,5073
19,lewis,analytic,13120
29,quine,analytic,7373
28,popper,analytic,4678
34,wittgenstein,analytic,9034
25,moore,analytic,3668
0,aristotle,aristotle,48777
32,smith,capitalism,11693
15,keynes,capitalism,3411
